In [4]:
import os
current_dir = os.getcwd()

# Define the Delta table path relative to the current directory
delta_table_path = os.path.join(current_dir, "Delta_Lake/")

movieLens_folder_path = 'MovieLens_32M'
delta_table_movieLens_path = delta_table_path+'MovieLens/'

imdb_folder_path = 'IMDB_datasets'
delta_table_imdb_path = f'{delta_table_path}IMDB/'

boxOffice_folder_path = 'BoxOffice'
delta_table_boxOffice_path = delta_table_path+'BoxOffice/'

Each .parquet file represents a partition of data (in case of partitioned data) or a set of rows for unpartitioned data.

.CRC files are automatically generated by Delta Lake during file writes and are used for quick integrity checks during reads.

In [1]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("LocalDeltaTable") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    

spark = configure_spark_with_delta_pip(builder).getOrCreate()


# MovieLens

In [ ]:
for filename in os.listdir(movieLens_folder_path):
    # Check if the file is a CSV
    if filename.endswith('.csv'):
        # Construct full file path
        file_path = os.path.join(movieLens_folder_path, filename)
        
        try:
            df = spark.read.format('csv').option('header', True).load(file_path)
            # df.show()

            df.write.format('delta').mode('overwrite').save(delta_table_movieLens_path+filename.removesuffix('csv'))
            
            
        except Exception as e:
            print(f"Error reading {filename}: {e}")

In [ ]:
# spark.read.format('delta').load(delta_table_movielens_path+'links').show()

# IMDB

In [5]:
from files_imbd import imbd_ingestion

imbd_ingestion()

for filename in os.listdir(imdb_folder_path):
    # Check if the file is a CSV
    if filename.endswith('.tsv'):
        # Construct full file path
        file_path = os.path.join(imdb_folder_path, filename)
        print(file_path)
        try:
            df = spark.read.format('csv').option('header', True).option('delimiter', '\t').load(file_path)
            # # df.show()

            df.write.format('delta').mode('overwrite').save(delta_table_imdb_path+filename.removesuffix('tsv'))
            
            
        except Exception as e:
            print(f"Error reading {filename}: {e}")

Downloaded: ./IMDB_datasets/name.basics.tsv.gz
Downloaded: ./IMDB_datasets/title.akas.tsv.gz
Downloaded: ./IMDB_datasets/title.basics.tsv.gz
Downloaded: ./IMDB_datasets/title.crew.tsv.gz
Downloaded: ./IMDB_datasets/title.episode.tsv.gz
Downloaded: ./IMDB_datasets/title.principals.tsv.gz
Downloaded: ./IMDB_datasets/title.ratings.tsv.gz
IMDB_datasets\name.basics.tsv
IMDB_datasets\title.akas.tsv
IMDB_datasets\title.basics.tsv
IMDB_datasets\title.crew.tsv
IMDB_datasets\title.episode.tsv
IMDB_datasets\title.principals.tsv
IMDB_datasets\title.ratings.tsv


# Box Office

In [5]:
def clean_column_names(df):
    for col_name in df.columns:
        new_name = col_name.replace(" ", "_").replace("%", "percent").replace("±", "plus_minus")  # Modify as needed
        df = df.withColumnRenamed(col_name, new_name)
    return df

In [7]:
from boxoffice import boxOffice_weekenly_ingestion

boxOffice_weekenly_ingestion()

for filename in os.listdir(boxOffice_folder_path):
    # Check if the file is a CSV
    if filename.endswith('.json'):
        # Construct full file path
        file_path = os.path.join(boxOffice_folder_path, filename)
        
        try:
            df = spark.read.format("json").option("multiline", "true").load(file_path)
            # df.show()
            df = clean_column_names(df)
            df.write.format('delta').mode('overwrite').option('mergeSchema', True).save(delta_table_boxOffice_path+filename.removesuffix('json'))
            
            
        except Exception as e:
            print(f"Error reading {filename}: {e}")

Box office data successfully saved to ./BoxOffice\box_office_data.json


In [6]:
import os
current_dir = os.getcwd()

delta_table_path = os.path.join(current_dir, 'Project 1/Data Management/Landing Zone/Persistent Zone/imbd/')


In [7]:
delta_table_path

'c:\\Users\\usuario\\Desktop\\FIB\\Ciència de Dades\\2n Quadrimestre\\BDM\\BDM\\Project 1/Data Management/Landing Zone/Persistent Zone/imbd/'

In [2]:
def getSpecificDayTable(delta_path, date):
    history = spark.sql(f"DESCRIBE HISTORY delta.`{delta_path}`").toPandas()
    timestamp = history[history["timestamp"].dt.strftime("%Y-%m-%d") == date]
    # print(timestamp)
    return str(timestamp.iloc[0]['timestamp'])

In [8]:
from datetime import datetime
delta_table_imdb_path = 'Project 1/Data Management/Landing Zone/Persistent Zone/imbd/'
time = getSpecificDayTable(delta_table_path+'imbd_title.ratings', '2025-04-04')
df = spark.read.format('delta').option('timestampAsOf', time).load(delta_table_path+'imbd_title.ratings').show()


+----------+-------------+--------+
|    tconst|averageRating|numVotes|
+----------+-------------+--------+
|tt35630701|          6.0|       6|
|tt35630702|          8.3|       6|
|tt35630703|          7.7|       6|
|tt35630707|          8.8|       9|
|tt35630708|          6.9|       7|
|tt35630715|          7.7|       6|
|tt35630719|          8.7|       6|
|tt35630728|          7.9|       7|
|tt35630731|          7.3|       6|
|tt35630734|          7.9|       7|
| tt3563080|          6.4|     147|
| tt3563100|          7.4|      76|
|tt35631016|          7.0|       6|
|tt35631042|          8.2|       6|
|tt35631072|          7.7|       7|
| tt3563108|          7.7|      30|
|tt35631096|          5.4|       7|
|tt35631127|          7.7|       6|
| tt3563116|          7.6|      12|
|tt35631246|          8.7|       6|
+----------+-------------+--------+
only showing top 20 rows



In [6]:
df.show()

+----------+-------------+--------+
|    tconst|averageRating|numVotes|
+----------+-------------+--------+
|tt35630701|          6.0|       6|
|tt35630702|          8.3|       6|
|tt35630703|          7.7|       6|
|tt35630707|          8.8|       9|
|tt35630708|          6.9|       7|
|tt35630715|          7.7|       6|
|tt35630719|          8.7|       6|
|tt35630728|          7.9|       7|
|tt35630731|          7.3|       6|
|tt35630734|          7.9|       7|
| tt3563080|          6.4|     147|
| tt3563100|          7.4|      76|
|tt35631016|          7.0|       6|
|tt35631042|          8.2|       6|
|tt35631072|          7.7|       7|
| tt3563108|          7.7|      30|
|tt35631096|          5.4|       7|
|tt35631127|          7.7|       6|
| tt3563116|          7.6|      12|
|tt35631246|          8.7|       6|
+----------+-------------+--------+
only showing top 20 rows



In [24]:
from datetime import datetime
time = getSpecificDayTable(delta_table_boxOffice_path+'box_office_data', '2025-03-29')
df = spark.read.format('delta').option('timestampAsOf', time).load(delta_table_boxOffice_path+'box_office_data')

In [ ]:
import os
import kagglehub
current_folder = os.getcwd()

# Download the dataset and store it in the current folder
path = kagglehub.dataset_download("grouplens/movielens-20m-dataset")

100%|██████████| 195M/195M [00:15<00:00, 13.2MB/s] 

Extracting files...


In [43]:
import shutil
path2 = os.path.join(current_folder, './ml-20m')
shutil.move(path, path2)

'c:\\Users\\usuario\\Desktop\\FIB\\Ciència de Dades\\2n Quadrimestre\\BDM\\BDM\\./ml-20m'